# Introducción

Se está siguiendo como referencia la guía [*Train a diffusion model*](https://huggingface.co/docs/diffusers/en/tutorials/basic_training#train-a-diffusion-model) de Hugging Face.

Para evitar la repetición de código y agilizar las partes iterativas del entrenamiento, se crea una clase con los hiperparámetros del modelo a crear.

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128
    train_batch_size = 8
    eval_batch_size = 16
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "fp16"
    seed = 42
    output_dir = "ddpm-cars-128"

    push_to_hub = False

config = TrainingConfig()

# Preparación

Esta fase se conforma de los siguientes pasos:

- Descargar el dataset
- Realizar la división 70-30.
- Normalizar imágenes

A continuación, se descarga el dataset (`tanganke/stanford_cars`)[https://huggingface.co/datasets/tanganke/stanford_cars] de Hugging Face.

In [ ]:
from datasets import load_dataset

config.dataset_name = "tanganke/stanford_cars"
dataset = load_dataset(config.dataset_name, split="train")

Se extrae una muestra del dataset.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 4, figsize=(16, 4))
for i, image in enumerate(dataset[:4]["image"]):
    axs[i].imshow(image)
    axs[i].set_axis_off()
fig.show()

Se define la función de preprocesamiento de las imágenes.

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

Se da una muestra de cómo quedaría una imagen después del preprocesamiento.

In [ ]:
def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}

dataset.set_transform(transform)

Se envuelve el dataset en un `DataLoader`, que permite cargar el dataset desde múltiples hilos y aleatorizar el orden de las imágenes empleadas.

In [ ]:
import torch

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

# Creación del modelo

En este ejemplo, se usará una UNet para la generación de imágenes. Se darán 3 canales de entrada y salida, cada uno correspondiendo a un color del espacio RGB.

In [ ]:
from diffusers import UNet2DModel

model = UNet2DModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=3,  # the number of input channels, 3 for RGB images
    out_channels=3,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)

He de notar algo importante acá. Recordando el funcionamiento de una UNet, la capa de *downsampling*, es decir, el *encoder*, va a generalizar características sobre la imagen que se esté tratando. De hecho, es una red neuronal convolucional (CNN), lo que ya debería ser familiar en este momento.

Lo definido en `block_out_channels=(128, 128, 256, 256, 512, 512)` corresponde a los kernels que se aplican en la entrada de cada bloque. La cantidad de elementos en la tupla son la cantidad de kernels aplicados, mientras que cada valor corresponde a la cantidad de kernels que se aplica en cada capa.

In [ ]:
sample_image = dataset[0]["images"].unsqueeze(0)
print("Input shape:", sample_image.shape)

print("Output shape:", model(sample_image, timestep=0).sample.shape)

El componente que se encarga de la adición y limpieza de ruido es el scheduler. El scheduler tiene dos funciones principales:

1. Se encarga de añadir ruido de manera proporcional siguiendo una cantidad de pasos dada. Diferentes schedulers usarán distribuciones diferentes para el ruido. En esta ocasión se usará [DDPMScheduler](https://huggingface.co/docs/diffusers/en/api/schedulers/ddpm), donde el ruido se distribuye de manera gaussiana y se ajusta en cada paso hacia adelante (fast-forward).
    
    Este scheduler se basa en cadenas de Markov. En las cadenas de Markov, el estado actual sólo depende del estado anterior, no de la secuencias que lo condujeron al estado actual. En este contexto, cada imagen ruidosa $x_i$ es un estado en la cadena, por lo que su estado depende sólo de la imagen anterior $x_{i-1}$. De este modo, a medida de que se agregan más pasos de ruidos, se transita por una cadena de estados donde el estado final es una imagen conformada por puro ruido.
2. Luego de la fase de encoding, el scheduler indica cuánto ruido debe eliminarse siguiendo una regla aprendida durante el entrenamiento, limpiando la imagen, generándola de este modo. De nuevo, se emplea el concepto de cadenas de Markov, donde el estado de una imagen con menor ruido $y_i$ es el resultado de una imagen anterior con más ruido $y_{i-1}$.

In [ ]:
import torch
from PIL import Image
from diffusers import DDPMScheduler

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
noise = torch.randn(sample_image.shape)
timesteps = torch.LongTensor([50])
noisy_image = noise_scheduler.add_noise(sample_image, noise, timesteps)

Image.fromarray(((noisy_image.permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0])

In [ ]:
import torch.nn.functional as F

noise_pred = model(noisy_image, timesteps).sample
loss = F.mse_loss(noise_pred, noise)

Para el entrenamiento, se utilizará el optimizador AdamW, el cual es una variante de Adam adaptada para realizar decaimiento del peso, lo que evita el sobreajuste y ayuda a regularizar mejor el modelo.

In [ ]:
from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

Cada 10 épocas, se guarda un grid de imágenes generadas con base a unas imágenes del dataset.

In [ ]:
from diffusers import DDPMPipeline
from diffusers.utils import make_image_grid
import os

def evaluate(config, epoch, pipeline):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.Generator(device='cpu').manual_seed(config.seed), # Use a separate torch generator to avoid rewinding the random state of the main training loop
    ).images

    # Make a grid out of the images
    image_grid = make_image_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")

Mediante la librería Accelerator se define todo el flujo de trabajo de la generación de imágenes. Se cargan los hiperparámetros definidos al inicio, y se crea un bucle donde se va a entrenar el modelo hasta las épocas dadas.

In [ ]:
from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
import os

def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True
            ).repo_id
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch["images"]
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device,
                dtype=torch.int64
            )

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                # Predict the noise residual
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                if config.push_to_hub:
                    upload_folder(
                        repo_id=repo_id,
                        folder_path=config.output_dir,
                        commit_message=f"Epoch {epoch}",
                        ignore_patterns=["step_*", "epoch_*"],
                    )
                else:
                    pipeline.save_pretrained(config.output_dir)

La función `notebook_launcher` se encarga de ejecutar el bucle de entrenamiento, repartiendo la carga entre el hardware disponible.

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

Se guarda el estado del optimizador y del scheduler para seguir entrenando el modelo más tarde.

In [ ]:
output_dir = config.output_dir

# Guardar el estado del optimizador
optimizer_state_path = os.path.join(output_dir, "optimizer_state.pt")
torch.save(optimizer.state_dict(), optimizer_state_path)

# Guardar el estado del scheduler
scheduler_state_path = os.path.join(output_dir, "scheduler_state.pt")
torch.save(lr_scheduler.state_dict(), scheduler_state_path)

print(f"Optimizer and scheduler states saved at {output_dir}")